In [9]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn

In [10]:
class ActionDataset(Dataset):
    def __init__(self,Transform=None):
        super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        self.transform = Transform
        self.lst=[]
        self.label=[]
        # self.append(Data,label)

    def __getitem__(self, index):
        sample = self.lst[index],self.label[index]
        if self.transform:
            sample=self.transform(sample)
        return sample
    
    def __len__(self):
        return self.leng
    
    def SetTrans(self,Transform):
        self.transform=Transform
    
    def append(self,Data):
        # super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # assert label==None,"Label must be give"
        kpscore = np.expand_dims(Data['kp_score'][0],axis=2)
        kp = Data['kp'][0]
        data = np.concatenate((kp,kpscore),axis=2)
        # data = np.expand_dims(data,axis=0)
        label = Data['label']
        self.lst.append(data)
        self.label.append(label)
        self.leng=len(self.lst)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data.astype(np.float32)),torch.tensor(label)

In [16]:
def Train_val(model=None,dataloader=None,epochs=None,lr=1e-4,lr_schedule=None):
    assert all(param is not None for param in [model,dataloader,epochs]),"All param must be give in: model,dataloader,epochs"
    device=torch.device('cuda')
    model.to(device)
    optim=torch.optim.Adam(model.parameters(),lr)
    criteria=nn.CrossEntropyLoss()
    # criteria=nn.BCELoss()
    for epoch in range(epochs):
        for i,(data,label) in enumerate (dataloader):
            output=model(data.to(device))
            label=label.to(device)
            # _,predict=torch.max(output,dim=1)
            loss = criteria(output,label)
            optim.zero_grad()
            loss.backward()
            optim.step()
            print(f'Epoch: {epoch}, step: {i}/{len(dataloader)}, Loss: {loss:.5f}')

In [12]:
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(ToTensor())
for data in datalst:
    actDataset.append(data)
# data

In [17]:
dataloader=DataLoader(dataset=actDataset,shuffle=True)
model=ModelSTGCN(3,2)
Train_val(model,dataloader,10)

Epoch: 0, step: 0/300, Loss: 1.28960
Epoch: 0, step: 1/300, Loss: 1.00683
Epoch: 0, step: 2/300, Loss: 0.77650
Epoch: 0, step: 3/300, Loss: 0.98086
Epoch: 0, step: 4/300, Loss: 0.61973
Epoch: 0, step: 5/300, Loss: 0.43845
Epoch: 0, step: 6/300, Loss: 1.11863
Epoch: 0, step: 7/300, Loss: 0.94405
Epoch: 0, step: 8/300, Loss: 0.92631
Epoch: 0, step: 9/300, Loss: 0.57284
Epoch: 0, step: 10/300, Loss: 0.48638
Epoch: 0, step: 11/300, Loss: 0.82558
Epoch: 0, step: 12/300, Loss: 0.92353
Epoch: 0, step: 13/300, Loss: 0.51953
Epoch: 0, step: 14/300, Loss: 0.51706
Epoch: 0, step: 15/300, Loss: 0.60638
Epoch: 0, step: 16/300, Loss: 0.82432
Epoch: 0, step: 17/300, Loss: 0.39549
Epoch: 0, step: 18/300, Loss: 1.05845
Epoch: 0, step: 19/300, Loss: 0.38261
Epoch: 0, step: 20/300, Loss: 0.38169
Epoch: 0, step: 21/300, Loss: 0.32370
Epoch: 0, step: 22/300, Loss: 1.01604
Epoch: 0, step: 23/300, Loss: 0.24156
Epoch: 0, step: 24/300, Loss: 0.37343
Epoch: 0, step: 25/300, Loss: 0.39336
Epoch: 0, step: 26/300

In [ ]:
# # for epoch in range(2):
# #     for i,data in enumerate (dataloader):
# #         print(f'Epoch: {epoch}, Step: {i+1}/{len(dataloader)}, label: {data[1]}')
# dataview=iter(dataloader)
# data = dataview.next()

In [19]:
testfile=pds.read_pickle('test.pkl')

In [24]:
testact=ActionDataset(ToTensor())
testact.append(testfile)

In [29]:
torch.expand_copy()

torch.Size([126, 17, 3])

In [31]:
model.eval()
with torch.no_grad():
    testdata,testlabel = testact[0]
    testdata=testdata.unsqueeze(0).to('cuda')
    outputtest = model(testdata)
    _,predict = torch.max(outputtest,1)
    print(predict,testlabel)

tensor([1], device='cuda:0') tensor(1)


In [ ]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset(datalst[0])
# dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
# n_sample=len(actDataset)
# epochs=10
# n_iter=round(n_sample/batch_size)
# for epoch in range(epochs-8):
#     for i,data in enumerate (dataloader):
#         if (i+1)% (batch_size)==0:
#             print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}, Input shap: {data.shape}')

In [ ]:
# import math
# math.ceil(len(actDataset)/batch_size)

In [ ]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset(datalst[0])
# dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
# n_sample=len(actDataset)
# n_iter=round(n_sample/batch_size)
# model = ModelSTGCN(3,5)
# lr = 1e-3
# optim = torch.optim.Adam(model.parameters(),lr)
# lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optim,[15,30],gamma=0.1)
# epochs = 10
# for epoch in range(epochs):
#     for i,data in enumerate (dataloader):
#         if (i+1)% (batch_size)==0:
#             print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}')